# K Nearest Neighbors Predictive Model - 2013
- Stop and Frisk Data

In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np
import os

In [2]:
# Dowloading stop and frisk data
os.system("curl -O http://www.nyc.gov/html/nypd/downloads/zip/analysis_and_planning/2013_sqf_csv.zip")
#os.system("mv nycb2010_16d.zip PUIDATA")
os.system("sudo apt-get install unzip")
#os.getenv("PUIDATA")
os.system("unzip 2013_sqf_csv.zip") # -d" + os.getenv("PUIDATA"))

0

In [3]:
sqf_raw_data = pd.read_csv("2013.csv")
sqf_df = pd.DataFrame(sqf_raw_data)

/Users/christianrosado/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (10,73,83) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
pd.set_option('display.max_columns', None)

In [5]:
sqf_df.head()

,year,pct,ser_num,datestop,timestop,recstat,inout,trhsloc,perobs,crimsusp,perstop,typeofid,explnstp,othpers,arstmade,arstoffn,sumissue,sumoffen,compyear,comppct,offunif,officrid,frisked,searched,contrabn,adtlrept,pistol,riflshot,asltweap,knifcuti,machgun,othrweap,pf_hands,pf_wall,pf_grnd,pf_drwep,pf_ptwep,pf_baton,pf_hcuff,pf_pepsp,pf_other,radio,ac_rept,ac_inves,rf_vcrim,rf_othsw,ac_proxm,rf_attir,cs_objcs,cs_descr,cs_casng,cs_lkout,rf_vcact,cs_cloth,cs_drgtr,ac_evasv,ac_assoc,cs_furtv,rf_rfcmp,ac_cgdir,rf_verbl,cs_vcrim,cs_bulge,cs_other,ac_incid,ac_time,rf_knowl,ac_stsnd,ac_other,sb_hdobj,sb_outln,sb_admis,sb_other,repcmd,revcmd,rf_furt,rf_bulg,offverb,offshld,forceuse,sex,race,dob,age,ht_feet,ht_inch,weight,haircolr,eyecolor,build,othfeatr,addrtyp,rescode,premtype,premname,addrnum,stname,stinter,crossst,aptnum,city,state,zip,addrpct,sector,beat,post,xcoord,ycoord,dettypCM,lineCM,detailCM
0,2013,1,2,1012013,452,1,I,T,6.0,GRAND LARCENY,1,P,Y,N,N,,N,,0,0,N,,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,Y,Y,N,Y,N,N,N,Y,N,N,N,N,N,N,Y,Y,N,N,N,N,N,N,N,861,861,Y,N,,S,,M,B,,41,5,9,200,BK,BR,M,,R,,,PLAT,,,FULTON STREET,CHURCH STREET,,MANHATTAN,,,1,E,,,981349,198508,CM,1,45
1,2013,1,3,1012013,315,1,I,T,15.0,GRAND LARCENY,3,V,Y,N,N,,N,,0,0,N,,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,861,861,N,N,V,S,,M,A,,32,6,0,150,BK,BR,T,,R,,,TRAIN STATION,,,FULTON STREET,CHURCH STREET,,MANHATTAN,,,1,E,,,981349,198508,CM,1,45
2,2013,1,7,1012013,300,1,I,T,10.0,ROBBERY,5,V,Y,N,N,,N,,0,0,N,,Y,Y,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,Y,N,N,N,N,N,Y,Y,N,N,N,N,Y,N,N,N,N,Y,N,N,N,861,861,Y,N,V,S,DS,M,B,,20,5,7,170,BK,BR,Z,,R,,,TRAIN STATION,,,FULTON STREET,CHURCH STREET,,MANHATTAN,,,1,E,,,981349,198508,CM,1,85
3,2013,1,11,1012013,1212,A,O,P,1.0,ROBBERY,1,P,Y,N,N,,N,,0,0,Y,,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,1,1,N,N,,,,M,B,,57,5,11,150,BK,BR,T,,R,,,,,,READE STREET,HUDSON STREET,,MANHATTAN,,,1,G,,,981698,200203,CM,1,85
4,2013,1,12,1012013,2300,A,O,P,5.0,BURGLARY,2,V,Y,N,N,,N,,0,0,Y,,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,1,1,N,N,,,,M,B,,31,6,0,200,BR,BR,M,,R,,,,,,MERCER STREET,PRINCE STREET,,MANHATTAN,,,1,H,,,984642,203280,CM,1,14


In [6]:
features = ['sex', 'race', 'age', 'ht_feet', 'ht_inch', 'weight', 'frisked'] 
clasifier = ['arstmade']

In [7]:
df1 = sqf_df.loc[:,'sex':'weight'].copy()

In [8]:
df2 = sqf_df.loc[:,'city'].copy()

In [9]:
df1.drop(['dob'],inplace=True, axis=1)
df1.head()

,sex,race,age,ht_feet,ht_inch,weight
0,M,B,41,5,9,200
1,M,A,32,6,0,150
2,M,B,20,5,7,170
3,M,B,57,5,11,150
4,M,B,31,6,0,200


In [10]:
df2.head()

0    MANHATTAN
1    MANHATTAN
2    MANHATTAN
3    MANHATTAN
4    MANHATTAN
Name: city, dtype: object

In [11]:
df3 = sqf_df.loc[:,'arstmade':'frisked'].copy()

In [12]:
df4 = sqf_df.loc[:,'arstmade':'frisked'].copy()

In [13]:
df3.drop(['arstoffn', 'sumissue', 'sumoffen', 'compyear', 'comppct', 'offunif', 'officrid', 'frisked'],inplace=True, axis=1)
df3.head()

,arstmade
0,N
1,N
2,N
3,N
4,N


In [14]:
df4.drop(['arstoffn', 'sumissue', 'sumoffen', 'compyear', 'comppct', 'offunif', 'officrid', 'arstmade'],inplace=True, axis=1)
df4.head()

,frisked
0,Y
1,N
2,Y
3,N
4,N


In [15]:
knn_df = pd.concat([df1, df2, df4, df3], axis=1)

In [16]:
knn_df.head()

,sex,race,age,ht_feet,ht_inch,weight,city,frisked,arstmade
0,M,B,41,5,9,200,MANHATTAN,Y,N
1,M,A,32,6,0,150,MANHATTAN,N,N
2,M,B,20,5,7,170,MANHATTAN,Y,N
3,M,B,57,5,11,150,MANHATTAN,N,N
4,M,B,31,6,0,200,MANHATTAN,N,N


In [17]:
knn_df.city.unique()

array(['MANHATTAN', 'BRONX', 'BROOKLYN', 'QUEENS', 'STATEN IS',
       'STATEN ISLAND'], dtype=object)

In [18]:
knn_df.sex.unique()

array(['M', 'F', 'Z'], dtype=object)

In [19]:
knn_df.race.unique()

array(['B', 'A', 'W', 'Q', 'U', 'Z', 'P', 'I'], dtype=object)

In [20]:
knn_df.arstmade.unique()

array(['N', 'Y'], dtype=object)

In [21]:
knn_df.frisked.unique()

array(['Y', 'N'], dtype=object)

In [22]:
knn_df.describe()

,ht_feet,ht_inch,weight
count,191851.000000,191851.000000,191851.000000
mean,5.196548,6.360311,170.139874
std,0.408309,3.433878,38.948476
min,3.000000,0.000000,0.000000
25%,5.000000,4.000000,150.000000
50%,5.000000,7.000000,170.000000
75%,5.000000,9.000000,180.000000
max,7.000000,11.000000,999.000000


In [23]:
knn_df.weight
for x in knn_df['weight']:
    if x < 100: 
        knn_df['weight'].replace(x, np.nan, inplace=True)
    if x > 300: 
        knn_df['weight'].replace(x, np.nan, inplace=True)

In [24]:
for x in knn_df['age']:
    if x > 99:
        knn_df['age'].replace(x, np.nan, inplace=True)
    if x < 10:
        knn_df['age'].replace(x, np.nan, inplace=True)

In [25]:
knn_df.head()

,sex,race,age,ht_feet,ht_inch,weight,city,frisked,arstmade
0,M,B,41.0,5,9,200.0,MANHATTAN,Y,N
1,M,A,32.0,6,0,150.0,MANHATTAN,N,N
2,M,B,20.0,5,7,170.0,MANHATTAN,Y,N
3,M,B,57.0,5,11,150.0,MANHATTAN,N,N
4,M,B,31.0,6,0,200.0,MANHATTAN,N,N


In [26]:
knn_df.describe()

,age,ht_feet,ht_inch,weight
count,171359.000000,191851.000000,191851.000000,190865.000000
mean,28.390350,5.196548,6.360311,169.470678
std,11.732643,0.408309,3.433878,28.533140
min,10.000000,3.000000,0.000000,100.000000
25%,20.000000,5.000000,4.000000,150.000000
50%,25.000000,5.000000,7.000000,170.000000
75%,34.000000,5.000000,9.000000,180.000000
max,99.000000,7.000000,11.000000,300.000000


In [27]:
knn_drop_nan = knn_df.copy()
knn_drop_nan.dropna(inplace=True)
knn_drop_nan.reset_index(drop=True)

,sex,race,age,ht_feet,ht_inch,weight,city,frisked,arstmade
0,M,B,41.0,5,9,200.0,MANHATTAN,Y,N
1,M,A,32.0,6,0,150.0,MANHATTAN,N,N
2,M,B,20.0,5,7,170.0,MANHATTAN,Y,N
3,M,B,57.0,5,11,150.0,MANHATTAN,N,N
4,M,B,31.0,6,0,200.0,MANHATTAN,N,N
5,M,W,57.0,5,7,200.0,MANHATTAN,N,N
6,M,W,31.0,5,6,160.0,MANHATTAN,N,N
7,M,Q,40.0,5,5,165.0,MANHATTAN,Y,Y
8,M,B,40.0,5,9,180.0,MANHATTAN,Y,N
9,M,U,25.0,5,11,190.0,MANHATTAN,Y,N


In [28]:
knn_drop_nan.describe()

,age,ht_feet,ht_inch,weight
count,170513.000000,170513.000000,170513.000000,170513.000000
mean,28.393084,5.194847,6.373250,169.441691
std,11.719956,0.404842,3.424163,28.445456
min,10.000000,3.000000,0.000000,100.000000
25%,20.000000,5.000000,4.000000,150.000000
50%,25.000000,5.000000,7.000000,170.000000
75%,34.000000,5.000000,9.000000,180.000000
max,99.000000,7.000000,11.000000,300.000000


In [29]:
knn_drop_nan.columns

Index([u'sex', u'race', u'age', u'ht_feet', u'ht_inch', u'weight', u'city',
       u'frisked', u'arstmade'],
      dtype='object')

In [30]:
knn_df_with_labels = knn_drop_nan.copy()

In [31]:
                                #['B', 'A', 'W', 'Q', 'U', 'Z', 'P', 'I']
knn_drop_nan.replace(['M', 'F', 'B', 'P', 'Z', 'Q', 'W', 'A', 'I', 'U', ' ', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'BRONX', 'STATEN IS', 'STATEN ISLAND', 'Y', 'N'], [0., 1., 0., 1., 2., 3., 4., 5., 6., 7., np.nan, 1., 2., 3., 4., 5., 5., 0., 1.], inplace=True)

In [32]:
#Normalized Data
for x in knn_drop_nan.columns:
    i = np.array(knn_drop_nan[x],dtype=float)
    normalized = (i-min(i))/(max(i)-min(i))
    knn_drop_nan.drop(x, axis=1)
    knn_drop_nan[x] = normalized
    print normalized


[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.          0.71428571  0.         ...,  0.          1.          0.        ]
[ 0.34831461  0.24719101  0.11235955 ...,  0.15730337  0.28089888
  0.46067416]
[ 0.5   0.75  0.5  ...,  0.75  0.5   0.5 ]
[ 0.81818182  0.          0.63636364 ...,  0.27272727  0.90909091  1.        ]
[ 0.5    0.25   0.35  ...,  0.5    0.375  0.425]
[ 0.25  0.25  0.25 ...,  0.25  0.    0.25]
[ 0.  1.  0. ...,  1.  1.  1.]
[ 1.  1.  1. ...,  0.  1.  1.]


In [33]:
knn_drop_nan.describe()

,sex,race,age,ht_feet,ht_inch,weight,city,frisked,arstmade
count,170513.000000,170513.000000,170513.000000,170513.000000,170513.000000,170513.000000,170513.000000,170513.000000,170513.000000
mean,0.044316,0.205317,0.206664,0.548712,0.579386,0.347208,0.309507,0.424079,0.928651
std,0.158606,0.251684,0.131685,0.101210,0.311288,0.142227,0.301818,0.494204,0.257408
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.112360,0.500000,0.363636,0.250000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.168539,0.500000,0.636364,0.350000,0.250000,0.000000,1.000000
75%,0.000000,0.428571,0.269663,0.500000,0.818182,0.400000,0.500000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [34]:
knn_drop_nan.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            172022, 172023, 172024, 172025, 172026, 172027, 172028, 172029,
            172030, 172031],
           dtype='int64', length=170513)

# Year 2013

In [35]:
knn_drop_nan.dropna(axis=0, inplace=True)

In [45]:
from sklearn.model_selection import train_test_split
X = knn_drop_nan.loc[:,:'weight']
y = knn_drop_nan.loc[:,'arstmade']
training, testing, train_label, test_label = train_test_split(X, y, test_size=0.20, random_state=42)

In [46]:
from sklearn.neighbors import KNeighborsClassifier
#training = knn_drop_nan.loc[:231096,:'frisked']
#train_label = knn_drop_nan.loc[:231096,'arstmade']
knn = KNeighborsClassifier()
knn.fit(training, train_label) 
#testing = knn_drop_nan.loc[231097:288870,:'frisked']
predicted = knn.predict(testing)

In [47]:
from sklearn.metrics import accuracy_score
#test_label = knn_drop_nan.loc[231097:288870,'arstmade']
acc = accuracy_score(test_label, predicted)

In [48]:
from sklearn.metrics import confusion_matrix
con_mat = confusion_matrix(test_label, predicted, [0, 1])

In [49]:
import math
total_accuracy = (con_mat[0, 0] + con_mat[1, 1]) / float(np.sum(con_mat))
class1_accuracy = (con_mat[0, 0] / float(np.sum(con_mat[0, :])))
class2_accuracy = (con_mat[1, 1] / float(np.sum(con_mat[1, :])))
print('Year 2013 - Arrested')
print(con_mat)
print('Total accuracy: %.5f' % total_accuracy)
print('Class1 accuracy: %.5f' % class1_accuracy)
print('Class2 accuracy: %.5f' % class2_accuracy)
print('Geometric mean accuracy: %.5f' % math.sqrt((class1_accuracy * class2_accuracy)))

Year 2013 - Arrested
[[   12  2453]
 [  110 31528]]
Total accuracy: 0.92485
Class1 accuracy: 0.00487
Class2 accuracy: 0.99652
Geometric mean accuracy: 0.06965
